In [1]:
# ============================================
#   暗闇迷路（盤面のみ）コンソール可視化
#   ◎ 全セル連結
#   ◎ 直線最大長 ≤ MAX_STRAIGHT
#   ◎ 2×2 通路禁止
#   ○ 分岐同士を再接続してループ適度に生成
# ============================================

import random
from dataclasses import dataclass
from typing import List, Tuple

# ---------- パラメータ ----------
SIZE          = 11     # 盤面の一辺長
MAX_STRAIGHT  = 3      # 通路が連続する最大セル数
LOOP_RATE     = 0.12   # 木構造にあとから追加するループの確率

# ---------- データ構造 ----------
@dataclass
class Maze:
    v: List[List[bool]]   # 垂直壁 True=壁あり between (x,y)-(x+1,y)
    h: List[List[bool]]   # 水平壁 True=壁あり between (x,y)-(x,y+1)

# ---------- 基本ユーティリティ ----------
def neigh(x: int, y: int):
    if x > 0:         yield 'L', x-1, y
    if x < SIZE-1:    yield 'R', x+1, y
    if y > 0:         yield 'U', x, y-1
    if y < SIZE-1:    yield 'D', x, y+1

def remove_wall(m: Maze, x: int, y: int, d: str):
    if d == 'L': m.v[y][x-1] = False
    if d == 'R': m.v[y][x]   = False
    if d == 'U': m.h[y-1][x] = False
    if d == 'D': m.h[y][x]   = False

def wall(m: Maze, x: int, y: int, d: str) -> bool:
    if d == 'L': return m.v[y][x-1]
    if d == 'R': return m.v[y][x]
    if d == 'U': return m.h[y-1][x]
    if d == 'D': return m.h[y][x]
    return True

def causes_open_square(m: Maze, x: int, y: int, d: str) -> bool:
    checks = []
    if d in ('L','R'):
        dx = -1 if d=='L' else 0
        for dy in (0,-1): checks.append((x+dx, y+dy))
    else:
        dy = -1 if d=='U' else 0
        for dx in (0,-1): checks.append((x+dx, y+dy))
    for cx, cy in checks:
        if 0 <= cx < SIZE-1 and 0 <= cy < SIZE-1:
            if (not m.v[cy][cx]   and not m.v[cy+1][cx] and
                not m.h[cy][cx]   and not m.h[cy][cx+1]):
                return True
    return False

# ---------- 1. 木構造 (直線≤MAX_STRAIGHT) ----------
def generate_tree() -> Maze:
    v = [[True]*(SIZE-1) for _ in range(SIZE)]
    h = [[True]*SIZE     for _ in range(SIZE-1)]
    visited = [[False]*SIZE for _ in range(SIZE)]
    sx, sy = random.randrange(SIZE), random.randrange(SIZE)
    stack: List[Tuple[int,int,str,int]] = [(sx, sy, '', 0)]
    visited[sy][sx] = True
    while stack:
        x, y, pd, run = stack[-1]
        nexts = [(d,nx,ny) for d,nx,ny in neigh(x,y) if not visited[ny][nx]]
        # 直線長制限
        if run >= MAX_STRAIGHT:
            nexts = [t for t in nexts if t[0] != pd]
        # 2×2 通路禁止
        nexts = [t for t in nexts if not causes_open_square(Maze(v,h), x, y, t[0])]
        if nexts:
            d, nx, ny = random.choice(nexts)
            remove_wall(Maze(v,h), x, y, d)
            visited[ny][nx] = True
            stack.append((nx, ny, d, run+1 if d==pd else 1))
        else:
            stack.pop()
    return Maze(v,h)

# ---------- 2. ループ追加 ----------
def add_loops(m: Maze):
    edges = []
    for y in range(SIZE):
        for x in range(SIZE-1):
            if m.v[y][x]: edges.append((x,y,'R'))
    for y in range(SIZE-1):
        for x in range(SIZE):
            if m.h[y][x]: edges.append((x,y,'D'))
    random.shuffle(edges)

    deg = [[0]*SIZE for _ in range(SIZE)]
    for y in range(SIZE):
        for x in range(SIZE-1):
            if not m.v[y][x]:
                deg[y][x]+=1; deg[y][x+1]+=1
    for y in range(SIZE-1):
        for x in range(SIZE):
            if not m.h[y][x]:
                deg[y][x]+=1; deg[y+1][x]+=1

    for x,y,d in edges:
        if random.random() > LOOP_RATE: continue
        nx, ny = (x+1,y) if d=='R' else (x,y+1)
        if deg[y][x] >= 3 or deg[ny][nx] >= 3: continue
        if causes_open_square(m, x, y, d):     continue
        remove_wall(m, x, y, d)
        deg[y][x]+=1; deg[ny][nx]+=1

# ---------- 3. 迷路生成 ----------
def generate_maze() -> Maze:
    while True:
        maze = generate_tree()
        add_loops(maze)
        if not causes_open_square(maze, 0,0,'R'):  # ざっくり再確認
            return maze

# ---------- 4. コンソール描画 ----------
def render(maze: Maze) -> str:
    lines = [' '+'_'*SIZE]
    for y in range(SIZE):
        row = ['|']
        for x in range(SIZE):
            ch = ' '
            if y==SIZE-1 or maze.h[y][x]: ch = '_' if ch==' ' else ch
            if x==SIZE-1 or maze.v[y][x]:
                row.append(ch+'|')
            else:
                row.append(ch+' ')
        lines.append(''.join(row))
    return '\n'.join(lines)

# ---------- 実行 ----------
if __name__ == "__main__":
    m = generate_maze()
    print(render(m))


 ___________
|   |_  |  _        | |
| |_  |_ _  |_ _| | | |
|   |  _  |    _ _| | |
| |_  |  _  |_  | |_  |
|_|   |  _ _ _  |_  | |
|  _|_ _| |   |    _| |
|_ _  |  _  |_ _ _ _ _|
|   |   |  _|  _  |_  |
| |_ _| |  _ _|  _|   |
| |   |_ _  |  _ _ _| |
|_ _|_ _ _ _|_|_ _ _ _|


In [3]:
# =====================================
#  2×2 通路絶対禁止版 ― 盤面可視化だけ
# =====================================

import random
from dataclasses import dataclass
from typing import List, Tuple

# ────────── 調整可能なパラメータ ──────────
SIZE          = 10     # 盤面の一辺長
MAX_STRAIGHT  = 3      # 通路の連続長上限
LOOP_RATE     = 0.12   # 木構造へ再接続する確率

# ────────── データ構造 ──────────
@dataclass
class Maze:
    v: List[List[bool]]   # 垂直壁 True=壁あり between (x,y)-(x+1,y)
    h: List[List[bool]]   # 水平壁 True=壁あり between (x,y)-(x,y+1)

# ────────── 基本ユーティリティ ──────────
def neighbors(x: int, y: int):
    if x > 0:        yield 'L', x-1, y
    if x < SIZE-1:   yield 'R', x+1, y
    if y > 0:        yield 'U', x, y-1
    if y < SIZE-1:   yield 'D', x, y+1

def wall_exists(m: Maze, x: int, y: int, d: str) -> bool:
    return m.v[y][x-1] if d=='L' else \
           m.v[y][x]   if d=='R' else \
           m.h[y-1][x] if d=='U' else \
           m.h[y][x]   # 'D'

def carve(m: Maze, x: int, y: int, d: str):
    if d == 'L': m.v[y][x-1] = False
    if d == 'R': m.v[y][x]   = False
    if d == 'U': m.h[y-1][x] = False
    if d == 'D': m.h[y][x]   = False

# 2×2 通路が完成するかの判定
def creates_open_square(m: Maze, x: int, y: int, d: str) -> bool:
    # carve 予定の壁の影響を受ける 2×2 ブロックの左上座標候補
    cands = []
    if d in ('L','R'):
        dx = -1 if d=='L' else 0
        for dy in (0, -1):
            cands.append((x+dx, y+dy))
    else:
        dy = -1 if d=='U' else 0
        for dx in (0, -1):
            cands.append((x+dx, y+dy))
    for cx, cy in cands:
        if 0 <= cx < SIZE-1 and 0 <= cy < SIZE-1:
            # 4 辺がすべて通路になるか確認
            v_left   = not (m.v[cy][cx]     if cx >=0         else True)
            v_right  = not (m.v[cy][cx+1]   if cx+1 < SIZE-1  else True)
            h_top    = not (m.h[cy][cx]     if cy >=0         else True)
            h_bottom = not (m.h[cy+1][cx]   if cy+1 < SIZE-1  else True)
            # carve 後をシミュレート
            if d=='L'  and cy in (y-1, y) and cx==x-1: v_left  = True
            if d=='R'  and cy in (y-1, y) and cx==x  : v_right = True
            if d=='U'  and cx in (x-1, x) and cy==y-1: h_top   = True
            if d=='D'  and cx in (x-1, x) and cy==y  : h_bottom= True
            if not v_left and not v_right and not h_top and not h_bottom:
                return True
    return False

# ────────── 1. 木構造（直線 ≤ MAX_STRAIGHT）──────────
def generate_tree() -> Maze:
    v = [[True]*(SIZE-1) for _ in range(SIZE)]
    h = [[True]*SIZE     for _ in range(SIZE-1)]
    visited = [[False]*SIZE for _ in range(SIZE)]

    sx, sy = random.randrange(SIZE), random.randrange(SIZE)
    stack: List[Tuple[int,int,str,int]] = [(sx, sy, '', 0)]
    visited[sy][sx] = True

    while stack:
        x, y, prev_d, run = stack[-1]
        nxt = [(d,nx,ny) for d,nx,ny in neighbors(x,y) if not visited[ny][nx]]

        if run >= MAX_STRAIGHT:
            nxt = [t for t in nxt if t[0] != prev_d]

        nxt = [t for t in nxt if not creates_open_square(Maze(v,h), x, y, t[0])]

        if nxt:
            d, nx, ny = random.choice(nxt)
            carve(Maze(v,h), x, y, d)
            visited[ny][nx] = True
            stack.append((nx, ny, d, run+1 if d==prev_d else 1))
        else:
            stack.pop()
    return Maze(v,h)

# ────────── 2. ループ追加（2×2厳守・次数≤3）──────────
def add_loops(m: Maze):
    deg = [[0]*SIZE for _ in range(SIZE)]
    for y in range(SIZE):
        for x in range(SIZE-1):
            if not m.v[y][x]:
                deg[y][x]+=1; deg[y][x+1]+=1
    for y in range(SIZE-1):
        for x in range(SIZE):
            if not m.h[y][x]:
                deg[y][x]+=1; deg[y+1][x]+=1

    cand = [(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]] + \
           [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(cand)

    for x,y,d in cand:
        if random.random() > LOOP_RATE: continue
        nx, ny = (x+1,y) if d=='R' else (x,y+1)

        if deg[y][x] >= 3 or deg[ny][nx] >= 3: continue
        if creates_open_square(m, x, y, d):    continue

        # 直線長を簡易チェック — carve 後に run が4を超えないかだけ検査
        if d == 'R':
            run = 1
            ix = x-1
            while ix>=0 and not m.v[y][ix]:   run+=1; ix-=1
            ix = x+1
            while ix<SIZE-1 and not m.v[y][ix]: run+=1; ix+=1
            if run > MAX_STRAIGHT: continue
        else:
            run = 1
            iy = y-1
            while iy>=0 and not m.h[iy][x]:   run+=1; iy-=1
            iy = y+1
            while iy<SIZE-1 and not m.h[iy][x]: run+=1; iy+=1
            if run > MAX_STRAIGHT: continue

        carve(m, x, y, d)
        deg[y][x]+=1; deg[ny][nx]+=1

# ────────── 3. 最終迷路生成 ──────────
def generate_maze() -> Maze:
    while True:
        m = generate_tree()
        add_loops(m)
        # 最終チェック: 2×2 通路が無いか
        if not any(
            not m.v[y][x] and not m.v[y+1][x] and
            not m.h[y][x] and not m.h[y][x+1]
            for y in range(SIZE-1) for x in range(SIZE-1)
        ):
            return m

# ────────── 4. コンソール描画 ──────────
def render(m: Maze) -> str:
    lines = [' '+'_'*SIZE]
    for y in range(SIZE):
        row=['|']
        for x in range(SIZE):
            ch=' '
            if y==SIZE-1 or m.h[y][x]: ch='_' if ch==' ' else ch
            if x==SIZE-1 or m.v[y][x]: row.append(ch+'|')
            else:                      row.append(ch+' ')
        lines.append(''.join(row))
    return '\n'.join(lines)

# ────────── 動作テスト ──────────
if __name__ == "__main__":
    random.seed()  # 必要に応じて固定
    maze = generate_maze()
    print(render(maze))


 __________
|  _  |     |   |  _|
|_  |  _| |_ _| |_  |
|  _| | |_  | |_  | |
|_ _ _  | | | |  _| |
|  _  |_ _| | |  _ _|
| | |_ _|  _|_ _    |
| |_ _   _|  _ _ _| |
|_ _  |_  |  _|  _  |
|  _|_  |_ _|   | | |
|_ _ _ _ _ _ _|_ _ _|


In [8]:
# ============================================
# 2×2 通路禁止 + 直線長 ≤ MAX_STRAIGHT
# 十字路 OK（次数上限なし）版
# ============================================

import random
from dataclasses import dataclass
from typing import List, Tuple

# -------- 調整可能パラメータ --------
SIZE          = 10     # 盤面の一辺
MAX_STRAIGHT  = 3      # 通路が連続する最大セル数
LOOP_RATE     = 0.15   # 木構造に再接続する確率 (0～1)

# -------- 迷路データ構造 --------
@dataclass
class Maze:
    v: List[List[bool]]  # 垂直壁 (y,x) True=壁あり between (x,y)-(x+1,y)
    h: List[List[bool]]  # 水平壁 (y,x) True=壁あり between (x,y)-(x,y+1)

# -------- 基本ユーティリティ --------
def neighbors(x: int, y: int):
    if x > 0:        yield 'L', x-1, y
    if x < SIZE-1:   yield 'R', x+1, y
    if y > 0:        yield 'U', x, y-1
    if y < SIZE-1:   yield 'D', x, y+1

def carve(m: Maze, x: int, y: int, d: str):
    if d=='L': m.v[y][x-1]=False
    if d=='R': m.v[y][x]  =False
    if d=='U': m.h[y-1][x]=False
    if d=='D': m.h[y][x]  =False

def wall(m: Maze, x: int, y: int, d: str)->bool:
    return m.v[y][x-1] if d=='L' else m.v[y][x] if d=='R' \
        else m.h[y-1][x] if d=='U' else m.h[y][x]          # 'D'

# ---- 2×2 通路ブロック出現判定 ----
def creates_open_square(m: Maze, x: int, y: int, d: str) -> bool:
    cand=[]
    if d in ('L','R'):
        dx=-1 if d=='L' else 0
        cand+=[(x+dx,y),(x+dx,y-1)]
    else:
        dy=-1 if d=='U' else 0
        cand+=[(x,y+dy),(x-1,y+dy)]
    for cx,cy in cand:
        if 0<=cx<SIZE-1 and 0<=cy<SIZE-1:
            if (not m.v[cy][cx] and not m.v[cy+1][cx] and
                not m.h[cy][cx] and not m.h[cy][cx+1]):
                return True
    return False

# ---- 1. 木構造生成 (直線≤MAX_STRAIGHT) ----
def generate_tree()->Maze:
    v=[[True]*(SIZE-1) for _ in range(SIZE)]
    h=[[True]*SIZE     for _ in range(SIZE-1)]
    visited=[[False]*SIZE for _ in range(SIZE)]
    sx,sy=random.randrange(SIZE),random.randrange(SIZE)
    stack=[(sx,sy,'',0)]
    visited[sy][sx]=True
    while stack:
        x,y,pd,run=stack[-1]
        nxt=[(d,nx,ny) for d,nx,ny in neighbors(x,y) if not visited[ny][nx]]
        if run>=MAX_STRAIGHT:
            nxt=[t for t in nxt if t[0]!=pd]
        nxt=[t for t in nxt if not creates_open_square(Maze(v,h),x,y,t[0])]
        if nxt:
            d,nx,ny=random.choice(nxt)
            carve(Maze(v,h),x,y,d)
            visited[ny][nx]=True
            stack.append((nx,ny,d,run+1 if d==pd else 1))
        else:
            stack.pop()
    return Maze(v,h)

# ---- 2. ループ追加 (十字路許可) ----
def add_loops(m:Maze):
    edges=[(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]+\
          [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(edges)
    for x,y,d in edges:
        if random.random()>LOOP_RATE: continue
        if creates_open_square(m,x,y,d):    continue
        # 直線長が破綻しないか簡易判定
        if d=='R':
            run=1
            ix=x-1
            while ix>=0 and not m.v[y][ix]: run+=1; ix-=1
            ix=x+1
            while ix<SIZE-1 and not m.v[y][ix]: run+=1; ix+=1
            if run>MAX_STRAIGHT: continue
        else:
            run=1
            iy=y-1
            while iy>=0 and not m.h[iy][x]: run+=1; iy-=1
            iy=y+1
            while iy<SIZE-1 and not m.h[iy][x]: run+=1; iy+=1
            if run>MAX_STRAIGHT: continue
        carve(m,x,y,d)

# ---- 3. 2×2 通路が一切無いか検証 ----
def has_open_square(m:Maze)->bool:
    return any(
        not m.v[y][x] and not m.v[y+1][x] and
        not m.h[y][x] and not m.h[y][x+1]
        for y in range(SIZE-1) for x in range(SIZE-1)
    )

# ---- 4. フル生成 ----
def generate_maze()->Maze:
    while True:
        m=generate_tree()
        add_loops(m)
        if not has_open_square(m):
            return m

# ---- 5. ASCII 描画 ----
def render(m:Maze)->str:
    lines=[' '+'_'*SIZE]
    for y in range(SIZE):
        row=['|']
        for x in range(SIZE):
            ch=' '
            if y==SIZE-1 or m.h[y][x]: ch='_' if ch==' ' else ch
            row.append(ch+'|' if x==SIZE-1 or m.v[y][x] else ch+' ')
        lines.append(''.join(row))
    return '\n'.join(lines)




In [12]:
# ---- 実行 ----
if __name__=="__main__":
    random.seed()          # 必要ならシード固定
    maze=generate_maze()
    print(render(maze))

 __________
|   |  _|   |    _  |
| |  _|  _|_ _|_  |_|
| |_|  _|  _ _  |_  |
|   | |   |   |_|   |
| | |_ _| | |_  | |_|
|_ _ _  | |_  | |_  |
|   |   |_  | |_ _ _|
| |_ _| |  _|_ _ _  |
|   |  _| | | |   | |
|_|_ _ _ _ _|_ _|_ _|


In [14]:
# =============================================================
# 2×2 通路禁止 & 直線通路 ≤ 4 マス 版（十字路 OK）
# =============================================================

import random
from dataclasses import dataclass
from typing import List, Tuple

# ───────── 調整用パラメータ ─────────
SIZE          = 10      # 盤面一辺
MAX_STRAIGHT  = 4       # 直線で続くセル数の上限
LOOP_RATE     = 0.15    # 木構造に後から追加するループ確率 (0-1)

# ───────── 迷路データ構造 ─────────
@dataclass
class Maze:
    v: List[List[bool]]  # 垂直壁 (y,x) True=壁あり between (x,y)-(x+1,y)
    h: List[List[bool]]  # 水平壁 (y,x) True=壁あり between (x,y)-(x,y+1)

# ───────── ヘルパー ─────────
def neigh(x: int, y: int):
    if x > 0:        yield 'L', x-1, y
    if x < SIZE-1:   yield 'R', x+1, y
    if y > 0:        yield 'U', x, y-1
    if y < SIZE-1:   yield 'D', x, y+1

def wall(m: Maze, x:int, y:int, d:str)->bool:
    return m.v[y][x-1] if d=='L' else \
           m.v[y][x]   if d=='R' else \
           m.h[y-1][x] if d=='U' else \
           m.h[y][x]                 # 'D'

def carve(m: Maze, x:int, y:int, d:str):
    if d=='L': m.v[y][x-1]=False
    if d=='R': m.v[y][x]  =False
    if d=='U': m.h[y-1][x]=False
    if d=='D': m.h[y][x]  =False

# 2×2 通路が出来るか判定
def makes_open_square(m: Maze, x:int, y:int, d:str) -> bool:
    # チェック対象の 2×2 ブロック左上候補
    cands=[]
    if d in ('L','R'):
        dx=-1 if d=='L' else 0
        for dy in (0,-1): cands.append((x+dx,y+dy))
    else:
        dy=-1 if d=='U' else 0
        for dx in (0,-1): cands.append((x+dx,y+dy))
    for cx,cy in cands:
        if 0<=cx<SIZE-1 and 0<=cy<SIZE-1:
            v0 = not m.v[cy][cx]
            v1 = not m.v[cy+1][cx]
            h0 = not m.h[cy][cx]
            h1 = not m.h[cy][cx+1]
            # carve 後を仮想反映
            if d=='L' and cx==x-1 and cy in (y-1,y): v0=True
            if d=='R' and cx==x   and cy in (y-1,y): v1=True
            if d=='U' and cy==y-1 and cx in (x-1,x): h0=True
            if d=='D' and cy==y   and cx in (x-1,x): h1=True
            if v0 and v1 and h0 and h1:
                return True
    return False

# ───────── 1. 木構造 (直線 ≤ MAX_STRAIGHT) ─────────
def generate_tree()->Maze:
    v=[[True]*(SIZE-1) for _ in range(SIZE)]
    h=[[True]*SIZE     for _ in range(SIZE-1)]
    visited=[[False]*SIZE for _ in range(SIZE)]
    sx,sy=random.randrange(SIZE),random.randrange(SIZE)
    stack=[(sx,sy,'',0)]
    visited[sy][sx]=True
    while stack:
        x,y,pd,run=stack[-1]
        nxt=[(d,nx,ny) for d,nx,ny in neigh(x,y) if not visited[ny][nx]]
        if run>=MAX_STRAIGHT:
            nxt=[t for t in nxt if t[0]!=pd]
        nxt=[t for t in nxt if not makes_open_square(Maze(v,h),x,y,t[0])]
        if nxt:
            d,nx,ny=random.choice(nxt)
            carve(Maze(v,h),x,y,d)
            visited[ny][nx]=True
            stack.append((nx,ny,d, run+1 if d==pd else 1))
        else:
            stack.pop()
    return Maze(v,h)

# ───────── 2. ループ追加（十字路許可） ─────────
def add_loops(m: Maze):
    edges=[(x,y,'R') for y in range(SIZE)   for x in range(SIZE-1) if m.v[y][x]]+\
          [(x,y,'D') for y in range(SIZE-1) for x in range(SIZE)   if m.h[y][x]]
    random.shuffle(edges)
    for x,y,d in edges:
        if random.random()>LOOP_RATE: continue
        # 直線長チェック
        if d=='R':
            run=1
            ix=x-1
            while ix>=0 and not m.v[y][ix]: run+=1; ix-=1
            ix=x+1
            while ix<SIZE-1 and not m.v[y][ix]: run+=1; ix+=1
            if run>MAX_STRAIGHT: continue
        else:
            run=1
            iy=y-1
            while iy>=0 and not m.h[iy][x]: run+=1; iy-=1
            iy=y+1
            while iy<SIZE-1 and not m.h[iy][x]: run+=1; iy+=1
            if run>MAX_STRAIGHT: continue
        if makes_open_square(m,x,y,d): continue
        carve(m,x,y,d)

# ───────── 3. 完全生成 & 最終 2×2 検査 ─────────
def generate_maze()->Maze:
    while True:
        maze=generate_tree()
        add_loops(maze)
        # 念のため最終スキャン
        if any(
            not maze.v[y][x] and not maze.v[y+1][x] and
            not maze.h[y][x] and not maze.h[y][x+1]
            for y in range(SIZE-1) for x in range(SIZE-1)
        ):
            continue
        return maze

# ───────── 4. コンソール描画 ─────────
def render(m:Maze)->str:
    lines=[' '+'_'*SIZE]
    for y in range(SIZE):
        row=['|']
        for x in range(SIZE):
            ch=' '
            if y==SIZE-1 or m.h[y][x]: ch='_' if ch==' ' else ch
            row.append(ch+'|' if x==SIZE-1 or m.v[y][x] else ch+' ')
        lines.append(''.join(row))
    return '\n'.join(lines)

# ───────── 実行例 ─────────
if __name__=="__main__":
    random.seed()  # 必要に応じて固定
    maze=generate_maze()
    print(render(maze))


 __________
|  _|  _ _    |_    |
| |  _|   | |  _ _| |
|_ _  | | | |_ _ _  |
|    _|   |_ _ _  | |
| |_  | |_ _ _  | |_|
| |_ _  | |  _|_  | |
| |   | | |  _  | | |
|  _| | |   | |  _  |
|_    |_| |  _    | |
|_ _|_ _ _|_ _ _|_ _|


In [15]:
# ---------------- 追加: 直線長を正確に評価 -----------------
def max_run_after_carve(m: Maze, x: int, y: int, d: str) -> int:
    """
    壁 (x,y,d) を壊した場合に出来る
    “横方向と縦方向で最長になる直線通路長” を返す。
    どちらか一方でも MAX_STRAIGHT を超えたら NG 判定に使う。
    """
    # 仮想的に壊した後の run を数える関数
    def run_len_horz(px: int, py: int):
        run = 1
        ix = px-1
        while ix >= 0 and not m.v[py][ix]:
            run += 1; ix -= 1
        ix = px
        while ix < SIZE-1 and not m.v[py][ix]:
            run += 1; ix += 1
        return run

    def run_len_vert(px: int, py: int):
        run = 1
        iy = py-1
        while iy >= 0 and not m.h[iy][px]:
            run += 1; iy -= 1
        iy = py
        while iy < SIZE-1 and not m.h[iy][px]:
            run += 1; iy += 1
        return run

    # carve 後に通路になる 2 セルの座標
    if d == 'R':
        cells = [(x, y), (x+1, y)]
    elif d == 'L':
        cells = [(x-1, y), (x, y)]
    elif d == 'D':
        cells = [(x, y), (x, y+1)]
    else:  # 'U'
        cells = [(x, y-1), (x, y)]

    max_run = 0
    for cx, cy in cells:
        max_run = max(max_run,
                      run_len_horz(cx, cy),
                      run_len_vert(cx, cy))
    return max_run
